In [1]:
TOKEN = 'YOUR_TOKEN'
CHAT_ID = 'YOUR_CHAT_ID'
import requests
# url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
# print(requests.get(url).json())
message = 'This is bot, hello world'
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={CHAT_ID}&text={message}"
r = requests.get(url)
print(r.json())


# url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={CHAT_ID}&text={}"
# r = requests.get(url)
# print(r.json())

{'ok': True, 'result': {'message_id': 1387, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727690910, 'text': 'This is bot, hello world'}}


In [2]:
#!pip install apscheduler

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import logging
from nsetools import Nse
from ta.trend import ADXIndicator
from datetime import datetime, timedelta
import pytz
from apscheduler.schedulers.blocking import BlockingScheduler


# Suppress warnings and unnecessary output
logging.getLogger("yfinance").setLevel(logging.ERROR)

# Initialize Nifty 500 stock symbols
nse = Nse()
nifty500 = pd.read_csv("ind_nifty500list.csv")
n500 = nifty500['Symbol']

# Download historical stock data
def get_historical_data(symbol, period="6mo", interval="1h"):
    try:
        stock = yf.download(f"{symbol}.NS", period=period, interval=interval, progress=False)
        if stock.empty:
            raise ValueError(f"No data found for {symbol}")
        return stock
    except Exception as e:
        logging.error(f"Error processing {symbol}: {str(e)}")
        return None

# Function to detect candlestick patterns and price action signals across the 6-month data
def detect_candlestick_patterns(stock_data):
    """
    Detects patterns across 6 months and returns the most recent signal.
    """
    stock_data['20 Hour EMA'] = stock_data['Close'].ewm(span=20, adjust=False).mean()
    stock_data['50 Hour EMA'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
    stock_data['Volume Average'] = stock_data['Volume'].rolling(window=50).mean()

    # Calculate ADX for trend strength
    adx_indicator = ADXIndicator(stock_data['High'], stock_data['Low'], stock_data['Close'], window=14)
    stock_data['ADX'] = adx_indicator.adx()

    signals = []
    
    # Iterate over rows to detect patterns
    for i in range(1, len(stock_data)):
        open_price = stock_data['Open'].iloc[i]
        close_price = stock_data['Close'].iloc[i]
        low_price = stock_data['Low'].iloc[i]
        high_price = stock_data['High'].iloc[i]
        volume = stock_data['Volume'].iloc[i]
        avg_volume = stock_data['Volume Average'].iloc[i]
        adx = stock_data['ADX'].iloc[i]

        # Identify if the stock is under increased volume
        volume_increased = volume > avg_volume * 1.5

        # Trend strength: Only consider trades where ADX > 25
        if adx > 25:
            # Bullish Patterns
            if close_price > stock_data['50 Hour EMA'].iloc[i] and volume_increased:
                if close_price > open_price and (high_price - low_price) > (2 * (close_price - open_price)):
                    signals.append({'Signal': 'Buy', 'Reason': 'Hammer', 'Current Price': close_price, 'Date': stock_data.index[i]})
                elif (close_price > stock_data['50 Hour EMA'].iloc[i] and
                      stock_data['Close'].iloc[i-1] < stock_data['50 Hour EMA'].iloc[i-1]):
                    signals.append({'Signal': 'Buy', 'Reason': 'Bullish Engulfing', 'Current Price': close_price, 'Date': stock_data.index[i]})
                elif close_price > open_price and (close_price - open_price) > 0.75 * (high_price - low_price):
                    signals.append({'Signal': 'Buy', 'Reason': 'Three White Soldiers', 'Current Price': close_price, 'Date': stock_data.index[i]})
            
            # Bearish Patterns
            elif close_price < stock_data['50 Hour EMA'].iloc[i] and volume_increased:
                if open_price > close_price and (open_price - close_price) > 0.75 * (high_price - low_price):
                    signals.append({'Signal': 'Sell', 'Reason': 'Bearish Engulfing', 'Current Price': close_price, 'Date': stock_data.index[i]})
                elif close_price < open_price and (open_price - close_price) > (high_price - low_price) * 2:
                    signals.append({'Signal': 'Sell', 'Reason': 'Evening Star', 'Current Price': close_price, 'Date': stock_data.index[i]})
                elif open_price > close_price and (close_price - open_price) > 0.75 * (high_price - low_price):
                    signals.append({'Signal': 'Sell', 'Reason': 'Three Black Crows', 'Current Price': close_price, 'Date': stock_data.index[i]})
                elif close_price < open_price and stock_data['Close'].iloc[i-1] > open_price:
                    signals.append({'Signal': 'Sell', 'Reason': 'Dark Cloud Cover', 'Current Price': close_price, 'Date': stock_data.index[i]})
    
    # Return only the most recent signal from the past data
    return signals[-1] if signals else None

# Function to filter signals generated within the last couple of days with timezone-awareness
def filter_recent_signals(signals, days=2):
    """
    Filters signals that are within the last 'days' number of days, considering timezone differences.
    """
    recent_signals = []
    threshold_date = datetime.now(pytz.timezone('Asia/Kolkata')) - timedelta(days=days)
    
    for signal in signals:
        if signal and signal['Date'].tz_convert('Asia/Kolkata') >= threshold_date:
            recent_signals.append(signal)
    
    return recent_signals

# Function to scan multiple stocks for signals
def scan_stocks(stock_symbols):
    results = []

    for symbol in stock_symbols:
        df = get_historical_data(symbol)
        if df is not None:
            signal = detect_candlestick_patterns(df)
            if signal:
                signal['Symbol'] = symbol
                results.append(signal)
    
    return results

#Execute the scan on Nifty 500 and filter the signals
scan_results = scan_stocks(n500)

#Filter only the recent signals generated within the last 2 days
recent_signals = filter_recent_signals(scan_results, days=2)

if recent_signals:
    for result in recent_signals:
        message = f"Stock: {result['Symbol']}, Signal: {result['Signal']}, Reason: {result['Reason']}, Date: {result['Date']}, Current Price: {result['Current Price']}"
        url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={CHAT_ID}&text={message}"
        r = requests.get(url)
        print(r.json())

else:
    message = "No significant recent signals found."
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={CHAT_ID}&text={message}"
    r = requests.get(url)
    print(r.json())





{'ok': True, 'result': {'message_id': 1388, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691059, 'text': 'Stock: 3MINDIA, Signal: Sell, Reason: Bearish Engulfing, Date: 2024-09-30 12:15:00 05:30, Current Price: 33600.0'}}
{'ok': True, 'result': {'message_id': 1389, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691059, 'text': 'Stock: APLAPOLLO, Signal: Buy, Reason: Three White Soldiers, Date: 2024-09-30 14:15:00 05:30, Current Price: 1585.050048828125'}}
{'ok': True, 'result': {'message_id': 1390, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691059, 'text': 'Stock: AJANTPHARM, Signal

{'ok': True, 'result': {'message_id': 1413, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691063, 'text': 'Stock: POWERINDIA, Signal: Buy, Reason: Hammer, Date: 2024-09-30 13:15:00 05:30, Current Price: 13950.349609375'}}
{'ok': True, 'result': {'message_id': 1414, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691063, 'text': 'Stock: HONAUT, Signal: Sell, Reason: Dark Cloud Cover, Date: 2024-09-30 14:15:00 05:30, Current Price: 49000.0'}}
{'ok': True, 'result': {'message_id': 1415, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691063, 'text': 'Stock: IDFC, Signal: Buy, Reason: Hammer, 

{'ok': True, 'result': {'message_id': 1436, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691069, 'text': 'Stock: POLICYBZR, Signal: Sell, Reason: Dark Cloud Cover, Date: 2024-09-30 14:15:00 05:30, Current Price: 1623.0'}}
{'ok': True, 'result': {'message_id': 1437, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691069, 'text': 'Stock: PCBL, Signal: Buy, Reason: Hammer, Date: 2024-09-30 14:15:00 05:30, Current Price: 579.2999877929688', 'entities': []}}
{'ok': True, 'result': {'message_id': 1438, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691070, 'text': 'Stock: PATANJALI, Signal: Se

{'ok': True, 'result': {'message_id': 1459, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691076, 'text': 'Stock: WHIRLPOOL, Signal: Buy, Reason: Hammer, Date: 2024-09-30 12:15:00 05:30, Current Price: 2295.89990234375'}}
{'ok': True, 'result': {'message_id': 1460, 'from': {'id': 7890892484, 'is_bot': True, 'first_name': 'Thoribot', 'username': 'ThoriSwingsbot'}, 'chat': {'id': 2020490005, 'first_name': 'Rohit', 'type': 'private'}, 'date': 1727691076, 'text': 'Stock: ZFCVINDIA, Signal: Sell, Reason: Dark Cloud Cover, Date: 2024-09-30 14:15:00 05:30, Current Price: 15874.7998046875'}}
